In [1]:
from pypalace import Config, Model, Domains, Boundaries, Solver, Simulation

In [2]:
''' path to Palace install ''' 
path_to_palace = "/Users/firasabouzahr/Desktop/AWSPalace/install/bin/palace-arm64.bin"

# Pocket Transmon

In [3]:
pocket_meshfile = "pocket_transmon.bdf"

In [4]:
Simulation.get_mesh_attributes(pocket_meshfile)

,Name,ID,Type
0,substrate,1,Volume
1,air,2,Volume
2,qubit_pad_top,3,Surface
3,qubit_pad_bottom,4,Surface
4,coupler,5,Surface
5,truncated_resonator,6,Surface
6,ground_plane,7,Surface
7,far_field,8,Surface


In [5]:
''' define config object (Problem) and Model '''
pocket_config = Config(Type="Electrostatic",Output="pocket_electro_output")
pocket_config.add_Model(pocket_meshfile) # no AMR, meshed finely already

''' Materials '''
silicon = Domains.Material([1],1.0,11.45,0.0) # silicon 
air = Domains.Material([2],1.0,1.0,0.0) # air
pocket_config.add_Domains(Materials=[silicon,air]) # add the materials

''' Boundary Conditions '''
## terminals ##
top_pad_terminal = Boundaries.Terminal(Index=1,Attributes=[3]) # top capacitor pad 
bottom_pad_terminal = Boundaries.Terminal(Index=2,Attributes=[4]) # bottom capacitor pad
coupler_terminal = Boundaries.Terminal(Index=3,Attributes=[5]) # qubit-res coupler
resonator_terminal = Boundaries.Terminal(Index=4,Attributes=[6]) # resonator coupler - we won't use this but must assign it something

## Ground ##
Grounds = Boundaries.Ground(Attributes=[7,8]) ## ground plane, far field

''' Boundary Postprocessing '''
top_pad_sf = Boundaries.Postprocessing_SurfaceFlux(Index=1,Attributes=[3],Type="Electric")
bottom_pad_sf = Boundaries.Postprocessing_SurfaceFlux(Index=2,Attributes=[4],Type="Electric")
coupler_pad_sf = Boundaries.Postprocessing_SurfaceFlux(Index=3,Attributes=[5],Type="Electric")
resonator_sf = Boundaries.Postprocessing_SurfaceFlux(Index=4,Attributes=[6],Type="Electric")

## add boundary conditions and boundary postprocessing
pocket_config.add_Boundaries(BCs=[top_pad_terminal,bottom_pad_terminal,coupler_terminal,Grounds],
                             Postprocessing=[top_pad_sf,bottom_pad_sf,coupler_pad_sf,resonator_sf])

''' electrostatic simulation and linear solver paramters ''' 
electro_params = Solver.Electrostatic(Save=3)

Linear_params = Solver.Linear(Type="BoomerAMG",
                              KSPType = "CG",
                              Tol = 1e-6, # make more stringent for better results
                              MaxIts = 25)
                              
pocket_config.add_Solver(Simulation=electro_params,
                     Order= 2, # second order solver
                     Linear=Linear_params)

''' save config '''
pocket_path_to_json = "pocket_transmon-electrostatic_sim.json"
pocket_config.save_config(pocket_path_to_json)

''' run the simulation '''
pocket_simulation = Simulation(path_to_palace,pocket_path_to_json)
pocket_simulation.run_palace(n=5) # 5 mpi processses

_____________     _______
_____   __   \____ __   /____ ____________
____   /_/  /  __ ` /  /  __ ` /  ___/  _ \
___   _____/  /_/  /  /  /_/  /  /__/  ___/
  /__/     \___,__/__/\___,__/\_____\_____/


--> Warning!
Output folder is not empty; program will overwrite content! (pocket_electro_output)
Git changeset ID: v0.13.0-509-g360de9ab
Running with 5 MPI processes
Device configuration: cpu
Memory configuration: host-std
libCEED backend: /cpu/self/xsmm/blocked

Added 62 elements in 1 iterations of local bisection for under-resolved interior boundaries
Added 16719 duplicate vertices for interior boundaries in the mesh
Added 34457 duplicate boundary elements for interior boundaries in the mesh
Finished partitioning mesh into 5 subdomains

Characteristic length and time scales:
 L₀ = 1.600e-03 m, t₀ = 5.337e-03 ns

Mesh curvature order: 1
Mesh bounding box:
 (Xmin, Ymin, Zmin) = (-8.000e-04, -8.000e-04, -5.000e-04) m
 (Xmax, Ymax, Zmax) = (+8.000e-04, +8.000e-04, +5.000e-04) m

Parallel 

# Xmon

In [6]:
xmon_meshfile = "xmon.bdf"

In [7]:
Simulation.get_mesh_attributes(xmon_meshfile)

,Name,ID,Type
0,substrate,1,Volume
1,air,2,Volume
2,qubit,3,Surface
3,claw,4,Surface
4,ground_plane,5,Surface
5,far_field,6,Surface


In [8]:
''' define config object (Problem) and Model '''
xmon_config = Config(Type="Electrostatic",Output="xmon_electro_output")
xmon_config.add_Model(xmon_meshfile) # no AMR, meshed finely already

''' Materials '''
silicon = Domains.Material([1],1.0,11.45,0.0) # silicon 
air = Domains.Material([2],1.0,1.0,0.0) # air
xmon_config.add_Domains(Materials=[silicon,air]) # add the materials

''' Boundary Conditions '''
## terminals ##
cross_terminal = Boundaries.Terminal(Index=1,Attributes=[3]) # qubit cross
claw_terminal = Boundaries.Terminal(Index=2,Attributes=[4]) # claw

## Ground ##
Grounds = Boundaries.Ground(Attributes=[5,6]) # ground plane, far field

''' Boundary Postprocessing '''
cross_sf = Boundaries.Postprocessing_SurfaceFlux(Index=1,Attributes=[3],Type="Electric")
claw_sf = Boundaries.Postprocessing_SurfaceFlux(Index=2,Attributes=[4],Type="Electric")

## add boundary conditions and boundary postprocessing
xmon_config.add_Boundaries(BCs=[cross_terminal,claw_terminal,Grounds],
                             Postprocessing=[cross_sf,claw_sf])

''' electrostatic simulation and linear solver paramters ''' 
electro_params = Solver.Electrostatic(Save=3)

Linear_params = Solver.Linear(Type="BoomerAMG",
                              KSPType = "CG",
                              Tol = 1e-6, # make more stringent for better results
                              MaxIts = 25) 
                              
xmon_config.add_Solver(Simulation=electro_params,
                       Order= 2, # second order solver
                       Linear=Linear_params)

''' save config '''
xmon_path_to_json = "xmon-electrostatic_sim.json"
xmon_config.save_config(xmon_path_to_json)

''' run the simulation '''
xmon_simulation = Simulation(path_to_palace,xmon_path_to_json)
xmon_simulation.run_palace(n=5) # 5 mpi processses

_____________     _______
_____   __   \____ __   /____ ____________
____   /_/  /  __ ` /  /  __ ` /  ___/  _ \
___   _____/  /_/  /  /  /_/  /  /__/  ___/
  /__/     \___,__/__/\___,__/\_____\_____/


--> Warning!
Output folder is not empty; program will overwrite content! (xmon_electro_output)
Git changeset ID: v0.13.0-509-g360de9ab
Running with 5 MPI processes
Device configuration: cpu
Memory configuration: host-std
libCEED backend: /cpu/self/xsmm/blocked

Added 76 elements in 1 iterations of local bisection for under-resolved interior boundaries
Added 4508 duplicate vertices for interior boundaries in the mesh
Added 10363 duplicate boundary elements for interior boundaries in the mesh
Finished partitioning mesh into 5 subdomains

Characteristic length and time scales:
 L₀ = 1.000e-03 m, t₀ = 3.336e-03 ns

Mesh curvature order: 1
Mesh bounding box:
 (Xmin, Ymin, Zmin) = (-4.000e-04, -4.000e-04, -5.000e-04) m
 (Xmax, Ymax, Zmax) = (+4.000e-04, +4.000e-04, +5.000e-04) m

Parallel Mes